# LG Pilot Demo

In [2]:
pip install transformers torch PyMuPDF python-docx python-pptx konlpy rank-bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 124.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 15.7 MB/s eta 0:00:00


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import os
from typing import Dict, List, Any

# PDF 처리
import fitz
# DOCX 처리
from docx import Document
# PPTX 처리
from pptx import Presentation
from konlpy.tag import Okt
from rank_bm25 import BM25Okapi

# ==============================================================================
# 1. 모델/토크나이저 초기화 및 환경 설정
# ==============================================================================

model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B"
model = None
tokenizer = None
okt = Okt() # KonLPy Okt 초기화

def initialize_llm(model_name: str):
    """LLM 모델과 토크나이저를 로드하고 전역 변수에 할당합니다."""
    global model, tokenizer
    try:
        print(f"Loading model: {model_name}...")
        # ⭐ GPU 환경에서 실행을 위해 device_map="auto"와 dtype="bfloat16" 사용
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            dtype=torch.bfloat16,
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        print("Model and Tokenizer loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")
        model, tokenizer = None, None
        print("FATAL: Model loading failed. LLM generation will be impossible.")

def tokenize_ko_en(text: str) -> List[str]:
    """한글은 형태소 분석(스테밍 포함), 영어는 소문자화 및 공백 분리하여 토큰 반환"""
    # 한글 형태소 분석
    ko_tokens = okt.morphs(text, stem=True)

    # 영어 및 기타 텍스트 처리 (형태소 분석 결과에 포함되지 않은 토큰)
    text_words = set(text.split())

    en_tokens = [
        token.lower()
        for token in text_words
        if token.lower() not in ko_tokens and token.isalnum() # 순수 영어 단어 추출 시 유효성 검사 추가
    ]
    return ko_tokens + en_tokens

In [7]:
# ==============================================================================
# 2. 문서 처리 함수
# ==============================================================================

def parse_pdf(file_path: str) -> List[Dict]:
    """PDF 파일을 파싱하여 페이지별 텍스트를 리스트로 반환합니다."""
    pages_data = []
    try:
        doc = fitz.open(file_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text = page.get_text("text")
            if text.strip():
                pages_data.append({'type': 'page', 'index': page_num + 1, 'content': text.strip()})
        doc.close()
    except Exception as e:
        print(f"PDF Parsing Error in {file_path}: {e}")
    return pages_data

def parse_pptx(file_path: str) -> List[Dict]:
    """PPTX 파일을 파싱하여 슬라이드별 텍스트를 리스트로 반환합니다."""
    slides_data = []
    try:
        prs = Presentation(file_path)
        for slide_num, slide in enumerate(prs.slides):
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.has_text_frame:
                    slide_text.append(shape.text)

            content = "\n".join(t.strip() for t in slide_text if t.strip())
            if content:
                 slides_data.append({'type': 'slide', 'index': slide_num + 1, 'content': content})
    except Exception as e:
        print(f"PPTX Parsing Error in {file_path}: {e}")
    return slides_data

def parse_docx(file_path: str) -> List[Dict]:
    """DOCX 파일을 파싱하여 단락별 텍스트를 리스트로 반환합니다."""
    paragraphs_data = []
    try:
        doc = Document(file_path)
        for para_num, paragraph in enumerate(doc.paragraphs):
            text = paragraph.text.strip()
            if text:
                paragraphs_data.append({'type': 'paragraph', 'index': para_num + 1, 'content': text})
    except Exception as e:
        print(f"DOCX Parsing Error in {file_path}: {e}")
    return paragraphs_data

def parse_folder_documents(folder_path: str) -> List[Dict[str, Any]]:
    """폴더 내의 지원되는 문서 파일들을 파싱하여 '문서 객체 리스트' 형태로 반환합니다."""
    all_documents_list: List[Dict[str, Any]] = []
    if not os.path.exists(folder_path):
        print(f"Error: Folder path does not exist: {folder_path}")
        return []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            extension = filename.split('.')[-1].lower()
            parsed_data: List[Dict] = []

            try:
                if extension == 'pdf':
                    parsed_data = parse_pdf(file_path)
                elif extension == 'pptx':
                    parsed_data = parse_pptx(file_path)
                elif extension == 'docx':
                    parsed_data = parse_docx(file_path)

                if parsed_data:
                    # 문서 단위 객체로 구성
                    all_documents_list.append({
                        'filename': filename,
                        'passages': parsed_data
                    })
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return all_documents_list

In [15]:
# ==============================================================================
# 3. RAG/BM25 및 LLM 관련 함수
# ==============================================================================

def call_llm_generate(prompt: str, max_new_tokens: int) -> str:
    """LLM을 호출하고 응답 텍스트만 반환합니다. 모델 로드 실패 시 예외를 발생시킵니다."""
    global model, tokenizer
    if model is None or tokenizer is None:
        raise RuntimeError("FATAL ERROR: LLM Model or Tokenizer failed to load. Cannot perform generation.")

    messages = [{"role": "user", "content": prompt}]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device) # 입력 텐서를 모델 디바이스로 이동

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    response_raw = tokenizer.decode(output[0], skip_special_tokens=True)

    tag = "</think>"
    if tag in response_raw:
        return response_raw
    else:
        # 모델이 프롬프트만 반환하는 경우, 전체를 반환
        return response_raw

def retrieve_and_augment_by_file(query: str, file_passages: List[Dict], file_name: str, N: int = 5):
    """
    특정 파일 내의 조각(Passages)에 대해서만 BM25 검색을 수행하고 1차 프롬프트를 구성합니다.
    """
    if not file_passages:
        return f"문서 '{file_name}'에는 내용이 없습니다.", None, None

    # 코퍼스 토큰화 및 BM25 인덱스 생성
    tokenized_corpus = [tokenize_ko_en(p['content']) for p in file_passages]
    bm25 = BM25Okapi(tokenized_corpus)

    # 질문 토큰화 및 점수 계산
    tokenized_query = tokenize_ko_en(query)
    doc_scores = bm25.get_scores(tokenized_query)

    # 관련성 높은 상위 N개 인덱스 추출
    top_n_indices = doc_scores.argsort()[::-1][:N]

    print("-" * 50)
    print(f"🔎 BM25 검색 시작 (문서: {file_name})")
    print("-" * 50)

    # 문서 내용 포맷팅
    document_parts = []

    for rank_idx, doc_index in enumerate(top_n_indices):
        passage_info = file_passages[doc_index]
        doc_content = passage_info['content'].replace('\n', ' ')
        document_parts.append(f"번호 {rank_idx + 1}. : {doc_content}")
        # 디버깅용 출력은 생략하고 프롬프트에 집중
        print(f"번호 {rank_idx + 1}. 페이지 {passage_info['index']} {doc_content[:200]}")

    document_formatted = "\n".join(document_parts)

    # 1차 프롬프트 템플릿: 가장 관련 깊은 문서 번호 선택 유도
    prompt_first_turn = f"""상황: {query}
{document_formatted}

저 문서들(번호 1~{len(top_n_indices)}) 중 현 상황과 가장 관련이 깊은 문서의 **번호 번호(숫자)**만 답변해 줘."""

    return prompt_first_turn, top_n_indices, file_name

def select_document_and_ask_sufficient_action(query: str, model_response_rank_raw: str, file_passages: List[Dict], top_n_indices: List[int], selected_doc_title: str) -> str:
    """
    모델 응답에서 번호를 추출하여 문서를 선택하고, 조치의 충분성을 묻는 2차 프롬프트를 생성합니다.
    """
    # 1. 모델 응답에서 번호 추출
    response_content = model_response_rank_raw.split('</think>')[-1]
    rank_match = re.search(r'\d+', response_content)
    selected_rank = int(rank_match.group(0)) if rank_match else 1

    # 2. 선택된 문서 정보 가져오기 (유효성 검사 및 인덱싱)
    if selected_rank > len(top_n_indices) or selected_rank <= 0:
        print(f"경고: 선택된 번호 번호({selected_rank})가 유효하지 않아 1위 문서로 대체합니다.")
        selected_rank = 1

    try:
        selected_doc_index = top_n_indices[selected_rank - 1]
        selected_passage = file_passages[selected_doc_index]
        selected_content_type = selected_passage['type']
        selected_index = selected_passage['index']
        selected_doc_content = selected_passage['content'].replace('\n', ' ')
    except IndexError:
        # 매우 드문 케이스, 이미 위에서 처리했으나 안전을 위해 추가
        print("치명적 오류: top_n_indices 인덱싱 실패. 첫 번째 문서로 대체.")
        selected_passage = file_passages[top_n_indices[0]]
        selected_content_type = selected_passage['type']
        selected_index = selected_passage['index']
        selected_doc_content = selected_passage['content'].replace('\n', ' ')

    # 3. 2차 프롬프트 템플릿 정의 및 포맷: 조치 답변 생성
    doc_identifier = f"{selected_content_type} {selected_index}"

    prompt_second_turn = f"""
상황: {query}

--- 선택된 문서 ({selected_doc_title}, {doc_identifier}) ---
{selected_doc_content}
--------------------------------

위 선택된 문서를 참고하여, 현 상황에 대해 **가장 적절한 조치**를 짧게 답변해 줘."""

    return prompt_second_turn

def create_rewriting_prompt(full_response: str) -> str:
    """다음 검색에 최적화된 쿼리를 생성하는 프롬프트를 만듭니다."""
    rewriting_prompt = f"""
다음은 이전 단계에서 '불충분'하다고 판단된 최종 조치 결과입니다.

--- 이전 조치 답변 ---
{full_response}
--------------------

이 답변의 내용을 기반으로, **현 상황을 해결하기 위해 다음 턴에 검색을 수행할 용도의 가장 핵심적이고 구체적인 질문 1개**만 생성해 주세요. 질문은 10단어 이내로 간결해야 합니다. (예: '압축기 교체 후 추가 점검 사항')
"""
    return rewriting_prompt

In [16]:
# ==============================================================================
# 4. 초기 설정 및 순차적 RAG 루프 실행
# ==============================================================================

if __name__ == "__main__":
    # 1. LLM 초기화
    initialize_llm(model_name)

    if model is None:
        exit() # 모델 로드 실패 시 종료

    # 2. 문서 파싱 설정
    documents_folder_path = "/content"
    documents_list = parse_folder_documents(documents_folder_path)

    if not documents_list:
        print("FATAL ERROR: No documents loaded. Please ensure files exist at the path.")
        exit()

    # 문서 순서 정의
    DOCUMENT_ORDER = [
        '문서_샘플_1.pdf',
        '문서_샘플_2.pdf',
        '문서_샘플_3.pdf',
    ]
    MAX_ITERATIONS = len(DOCUMENT_ORDER)

    # 초기 쿼리 설정
    initial_query = """CH21 에러 발생으로 실내기 운전 불가 에러 최초발생시점 : 2025.06.03, 에러 최신발생시점 : 2025.07.08\n누적발생일수 : 33, ch21 : 106, ch26: 0, ch29 :0, 백업inv1 : 정상, 백업inv2 : 정상"""
    # initial_query = input()
    current_query = initial_query
    iteration_history = []

    print(f"\n🚀 Initial Query: {initial_query}")

    # 3. 순차적 RAG 루프 실행
    for i, doc_name_to_search in enumerate(DOCUMENT_ORDER):
        print("\n" + "#"*80)
        print(f"🔄 ITERATION {i+1}/{MAX_ITERATIONS}")
        # print(f"   현재 검색 쿼리: {current_query}")
        print("#"*80)

        # [문서 단위 처리] 문서 객체 찾기
        document_unit: Dict[str, Any] = next((doc for doc in documents_list if doc['filename'] == doc_name_to_search), {})

        if not document_unit or not document_unit.get('passages'):
            print(f"Warning: Document '{doc_name_to_search}' not found or has no content. Skipping.")
            continue

        file_passages: List[Dict] = document_unit['passages']

        # A. 1차 단계: BM25 검색 및 문서 번호 선택 프롬프트 생성
        prompt_to_get_rank, top_n_indices, selected_doc_title = retrieve_and_augment_by_file(
            query=current_query,
            file_passages=file_passages,
            file_name=doc_name_to_search,
            N=5
        )

        # B. 1차 모델 호출: 관련 문서 번호(숫자) 답변 받기
        try:
            rank_response_raw = call_llm_generate(prompt_to_get_rank, max_new_tokens=20)
            rank_response = rank_response_raw.split('</think>')[-1].strip()
        except RuntimeError as e:
            print(f"LLM Call Error: {e}")
            break

        # print(f"\n💡 LLM 선택 번호: {rank_response}")

        # C. 2차 단계: 선택된 문서로 최종 조치 프롬프트 생성
        prompt_to_get_action = select_document_and_ask_sufficient_action(
            query=current_query,
            model_response_rank_raw=rank_response_raw,
            file_passages=file_passages,
            top_n_indices=top_n_indices,
            selected_doc_title=selected_doc_title
        )

        # D. 2차 모델 호출: 최종 조치 답변 받기
        try:
            action_response_raw = call_llm_generate(prompt_to_get_action, max_new_tokens=128)
            action_response = action_response_raw.split('</think>')[-1].strip()
        except RuntimeError as e:
            print(f"LLM Call Error: {e}")
            break

        print("\n" + "="*80)
        print(f"➡️ ITERATION {i+1} 답변: {action_response}")
        print("="*80)

        # E. 히스토리 저장 및 다음 턴 준비
        iteration_history.append({
            'turn': i + 1,
            'doc_name': doc_name_to_search,
            'query_for_search': current_query,
            'action_response': action_response
        })

        is_last_iteration = (i + 1) == MAX_ITERATIONS

        if is_last_iteration:
            print("\n✅ 모든 문서를 검색 완료했습니다. 반복을 종료합니다.")
            break

        # 다음 턴을 위한 새로운 검색 쿼리 생성
        rewriting_prompt = create_rewriting_prompt(action_response)
        print("\n🔍 새로운 검색 쿼리 생성을 위해 LLM 호출...")
        try:
            # 새로운 쿼리 생성 (max_new_tokens=20)
            new_query_response_raw = call_llm_generate(rewriting_prompt, max_new_tokens=20)
            # 모델 응답에서 질문 부분만 깔끔하게 추출
            current_query = new_query_response_raw.split("</think>")[-1]
        except RuntimeError as e:
            print(f"LLM Call Error during query rewriting: {e}")
            break

        print(f"새로운 검색 쿼리: {current_query}")
        current_query+=initial_query

Loading model: LGAI-EXAONE/EXAONE-4.0-1.2B...
Model and Tokenizer loaded successfully.

🚀 Initial Query: CH21 에러 발생으로 실내기 운전 불가 에러 최초발생시점 : 2025.06.03, 에러 최신발생시점 : 2025.07.08
누적발생일수 : 33, ch21 : 106, ch26: 0, ch29 :0, 백업inv1 : 정상, 백업inv2 : 정상

################################################################################
🔄 ITERATION 1/3
################################################################################
--------------------------------------------------
🔎 BM25 검색 시작 (문서: 문서_샘플_1.pdf)
--------------------------------------------------
번호 1. 페이지 13 12 / 61 LGE Internal Use Only 1. 압축기점검이력이존재합니다. TMS AI 고장예측가이드북 압축기점검 ※ 압축기점검테이블포맷예) 해당시스템(실외기)의일단위실외기운전이력(모드, 최대주파수운전이력) 및이에매칭되는압축기주요에러발생이력에대한정보를제공. ①: 최초발생시점: - 일단위점검판정에해당하는압축기에러의최초발생시점 ②: 최신발생시점: -
번호 2. 페이지 10 9 / 61 LGE Internal Use Only TMS AI 고장예측가이드북 압축기점검 ※ 압축기점검이란무엇인가요? 압축기점검 ①. 압축기에러: ch21, ch26, ch29의에러가일단위정상적인사용을저해하는수준으로지속발생하는경우를점검으로판정 ②. 백업이력체크: 데이터수집일최종일기준압축기가백업상태로유지된경우를점검으로판정 ③. 리포트(상세페이지)에서제공하는
번호 3. 페이지 23 22 /

# Curation